In [1]:
import scipy
import skimage
import sys
from skimage import io
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd
from numpy import matlib
import seaborn as sns
import math
import os
import scipy.optimize
import scipy.io as sio
from scipy import stats
import scipy.ndimage
import csv
# import h5py
sns.set(font_scale=1.5)
sns.set_style("white")


In [2]:
expts = [["/230323_bFB66_Tun_lysis", "/250626_bFB66_tun_lysis"],["/230317_bFB69_Tun_Lysis", "/230322_bFB69_Tun_lysis", "/240729_bFB69_Tun_lysis"]]
scene_nums = [[6,3],[3, 6, 3]]
len_times = [[26,26],[19, 26, 26]]
tperts = [[5.0],[5.0]]
starting_points = [[np.array([11, 11, 9, 11, 12, 11]), np.array([6,10,6])],[np.array([2,6,1]), np.array([6,3,2,1,1,1]), np.array([1,1,1])]]
strain_id = ['WT', r'$\Delta ponA$']
tstep = 5.0
out_name='ponA_lim'
pos=[0.1,0.6]

In [3]:
path = '/Volumes/data_ssd1/Rojas_Lab/data'
out_path = '/Users/barber.527/Documents/GitHub/Rojas_lab_drafts/outputs'

total_cells = []
lysed_cells = []
# Importing the data.

for ind in range(len(expts)):
    expt_ids=expts[ind]
    total_cells.append(np.nan * np.ones([np.sum(scene_nums[ind]), np.amax(len_times[ind])]))
    lysed_cells.append(np.nan * np.ones([np.sum(scene_nums[ind]), np.amax(len_times[ind])]))
    scene_tracker=0
    for expt_num in range(len(expt_ids)):
        for scene in range(scene_nums[ind][expt_num]):
            for timepoint in range(starting_points[ind][expt_num][scene],
                                   len_times[ind][expt_num] + 1):  # Note that in the filenames, time counts from 1, not zero
                #         print(timepoint)
                # Finding out the number of cells in each scene
                with open(path + expt_ids[expt_num] + '/annotated_images/s{0}'.format(
                        str(scene + 1).zfill(3)) + expt_ids[expt_num] + '_s{0}_t{1}_all_cells.csv'.format(str(scene + 1).zfill(3),
                                                                                               str(timepoint).zfill(
                                                                                                       4))) as csv_file:
                    csv_reader = csv.reader(csv_file, delimiter=',')
                    line_count = 0
                    for row in csv_reader:
                        #                 temp_out.append(row) # Data should be saved in UTF-8 format
                        line_count += 1
                    total_cells[ind][scene_tracker+scene, timepoint - 1] = line_count - 1
                # number of lysed cells
                with open(path + expt_ids[expt_num] + '/annotated_images/s{0}'.format(
                        str(scene + 1).zfill(3)) + expt_ids[expt_num]+ '_s{0}_t{1}_lysed_cells.csv'.format(str(scene + 1).zfill(3),
                                                                                                 str(timepoint).zfill(
                                                                                                         4))) as csv_file:
                    csv_reader = csv.reader(csv_file, delimiter=',')
                    line_count = 0
                    for row in csv_reader:
                        #                 temp_out.append(row) # Data should be saved in UTF-8 format
                        line_count += 1
                    lysed_cells[ind][scene_tracker+scene, timepoint - 1] = line_count - 1
        scene_tracker+=scene_nums[ind][expt_num]
# Next, we plot the average traces for all experiments.
frac_lysed=[]
for ind in range(len(expts)):
    frac_lysed.append(lysed_cells[ind]*100.0 / total_cells[ind])
    for row in range(lysed_cells[ind].shape[0]):
        frac_lysed[ind][row,:np.nonzero(~np.isnan(frac_lysed[ind][row,:]))[0][0]]=0


In [35]:
cols=['frac lysed', 'strain']
stat_vals=pd.DataFrame(columns=cols)
for ind in range(len(expts)):
    for scene in range(len(frac_lysed[ind])):
        if not np.isnan(frac_lysed[ind][scene][-1]):
            temp_df=pd.DataFrame(columns=cols, data=[[frac_lysed[ind][scene][-1],strain_id[ind]]])
            stat_vals=pd.concat([stat_vals,temp_df])

In [41]:
strains = stat_vals.strain.unique()
scipy.stats.ttest_ind(stat_vals[stat_vals.strain==strains[0]]['frac lysed'], stat_vals[stat_vals.strain==strains[1]]['frac lysed'])

Ttest_indResult(statistic=-6.779356578852532, pvalue=4.425726426670611e-06)